In [1]:
from sklearn.datasets import fetch_20newsgroups
import spacy
import gensim

In [2]:
newsgroups_train = fetch_20newsgroups('/content/drive/My Drive/Datasets',subset='train',
                                     remove=('headers', 'footers', 'quotes'))

In [3]:
newsgroups_test = fetch_20newsgroups('/content/drive/My Drive/Datasets',subset='test',
                                     remove=('headers', 'footers', 'quotes'))

In [4]:
import pprint
for i in range(2):
  print(newsgroups_train.data[i])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. T

In [5]:
len(newsgroups_train.data)

11314

In [6]:
import re
def preprocess(text):
  m_text = re.sub(r'\s+',' ',text)
  m_text = m_text.strip()
  m_text = re.sub(r'\S*@\S*' , '' ,m_text)
  m_text = re.sub(r'https?://\S+|www\.\S+','',m_text)
  m_text = re.sub(r'[^a-zA-Z_\s+]','',m_text)
  return m_text

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')
def tokenizer(text):
    m_text = preprocess(text)
    doc = nlp(m_text)
    doc = [token for token in doc if not (token.is_punct or token.is_stop or len(token.text) < 3 or len(token.text)>12) ]
    doc = [token.lemma_ if token.pos_ is not 'PRON' else token.lower_ for token in doc]
    return doc

In [8]:
train_data = [tokenizer(txt) for txt in newsgroups_train['data']]
test_data =  [tokenizer(txt) for txt in newsgroups_test['data']]

In [9]:
for i in range(5):
  print(train_data[i])

['wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'Bricklin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'email']
['fair', 'number', 'brave', 'soul', 'upgrade', 'clock', 'oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detail', 'experience', 'procedure', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'adapter', 'heat', 'sink', 'hour', 'usage', 'day', 'floppy', 'disk', 'floppy', 'especially', 'request', 'summarize', 'day', 'add', 'network', 'knowledge', 'base', 'clock', 'upgrade', 'answer', 'poll', 'thank']
['folk', 'mac', 'plus', 'finally', 'give', 'ghost', 'weekend', 'start', 'life', 'way', 'sooo', 'market', 'new', 'machine', 'bit', 'sooner', 'intend', 'look', 'pick', 'powerbook', 'maybe', 'bunch', 'question', 'hopefully', 'somebody', 'answer', 'anybody', 'kn

In [11]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
def bigrams(data):
    bigram = Phrases(data)
    bigram_model = Phraser(bigram)
    return bigram_model

bigram_model = bigrams(train_data)

In [13]:
train_bigram = [bigram_model[doc] for doc in train_data]
test_bigram = [bigram_model[doc] for doc in test_data]

In [18]:
for i in range(5):
  print(train_bigram[i])

['wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport_car', 'look', 'late', 'early', 'call', 'Bricklin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'email']
['fair', 'number', 'brave', 'soul', 'upgrade', 'clock_oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detail', 'experience', 'procedure', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'adapter', 'heat_sink', 'hour', 'usage', 'day', 'floppy_disk', 'floppy', 'especially', 'request', 'summarize', 'day', 'add', 'network', 'knowledge', 'base', 'clock', 'upgrade', 'answer', 'poll', 'thank']
['folk', 'mac', 'plus', 'finally', 'give', 'ghost', 'weekend', 'start', 'life', 'way', 'sooo', 'market', 'new', 'machine', 'bit', 'sooner', 'intend', 'look', 'pick', 'powerbook', 'maybe', 'bunch', 'question', 'hopefully', 'somebody', 'answer', 'anybody_know', 'dirt', 'r

In [21]:
from gensim.corpora import Dictionary
dictionary = Dictionary(documents= train_bigram, prune_at=None)
dictionary.filter_extremes(no_below=5,no_above= .3 ,keep_n=None)  # use Dictionary to remove un-relevant tokens
dictionary.compactify()

In [22]:
bow_train_data = [dictionary.doc2bow(doc) for doc in train_bigram]
bow_test_data = [dictionary.doc2bow(doc) for doc in test_bigram]

In [23]:
bow_train_data[3]

[(49, 1), (105, 1), (152, 1), (153, 1)]

In [232]:
from gensim.models import LdaMulticore
num_topics = 20

LDA = LdaMulticore(
    corpus=bow_train_data, num_topics=num_topics, id2word=dictionary, eval_every=None, passes=20, batch=True
    ,per_word_topics = True )

In [233]:
top_topics = LDA.top_topics(bow_train_data ,topn =7)

In [234]:
import pickle
pickle.dump(bow_train_data, open('/content/drive/My Drive/LDA/bow_train_corpus.pkl', 'wb'))
pickle.dump(bow_test_data, open('/content/drive/My Drive/LDA/bow_test_corpus.pkl', 'wb'))
LDA.save('/content/drive/My Drive/LDA/model.gensim')
dictionary.save('/content/drive/My Drive/LDA/gensim_dictionary.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [235]:
import pyLDAvis.gensim
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(LDA, bow_train_data, dictionary)

In [236]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.121884  0.023791       1        1  14.085595
6      0.116713  0.095764       2        1   9.455368
13    -0.163565 -0.016702       3        1   9.098508
15     0.135295  0.146413       4        1   7.555155
2     -0.027824 -0.075616       5        1   6.491548
14     0.033861  0.112129       6        1   6.287771
18    -0.118362  0.031695       7        1   5.378445
8      0.195754  0.023341       8        1   5.117361
4     -0.121375  0.017153       9        1   4.233367
9     -0.068620  0.035472      10        1   4.134365
11     0.006876  0.003684      11        1   3.710459
7     -0.045866  0.035295      12        1   3.694326
19     0.093599 -0.033026      13        1   3.540962
10    -0.062152  0.052490      14        1   3.166968
0      0.101796  0.082694      15        1   2.675127
5     -0.105097 -0.131298      16        1   2.663908
1      0.186572 -0.305315      17        1   2.574464
3     -0.024929 -0.103835      18        1   2.530813
17    -0.103908 -0.033184      19        1   1.956956
12     0.093118  0.039054      20        1   1.648544, topic_info=       Term         Freq        Total Category  logprob  loglift
62           2894.000000  2894.000000  Default  30.0000  30.0000
679           990.000000   990.000000  Default  29.0000  29.0000
210    file  1754.000000  1754.000000  Default  28.0000  28.0000
678           804.000000   804.000000  Default  27.0000  27.0000
1455   game  1047.000000  1047.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
62             42.219467  2894.010010  Topic20  -5.6513  -0.1222
172    tell    38.140350  1652.825073  Topic20  -5.7529   0.3363
300    work    38.740814  2630.372559  Topic20  -5.7373  -0.1127
112     new    36.792881  1682.729614  Topic20  -5.7889   0.2824
105    like    37.753231  3402.296875  Topic20  -5.7631  -0.3959

[1610 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
62         1  0.007256              
62         2  0.025570              
62         3  0.083621              
62         4  0.013131              
62         5  0.006220              
...      ...       ...           ...
567       12  0.090300     yesterday
567       19  0.024627     yesterday
10733      3  0.140002  young_people
10733      7  0.031112  young_people
10733     12  0.808901  young_people

[7708 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 7, 14, 16, 3, 15, 19, 9, 5, 10, 12, 8, 20, 11, 1, 6, 2, 4, 18, 13])

In [237]:
for i in range(10):
  print(top_topics[i])

([(0.013780172, 'post'), (0.010773995, 'people'), (0.008151971, 'like'), (0.007919357, 'group'), (0.0074083675, 'think'), (0.0072529586, 'right'), (0.0062818513, 'know')], -1.2682225647518988)
([(0.01671948, 'entry'), (0.014209497, 'file'), (0.014061238, 'program'), (0.011123775, 'line'), (0.010171141, 'problem'), (0.008263295, 'use'), (0.006917783, 'system')], -1.2712718974767643)
([(0.019815465, 'game'), (0.015184599, 'team'), (0.012752844, 'year'), (0.010867448, 'play'), (0.010864883, 'good'), (0.009931654, 'player'), (0.008976983, 'think')], -1.3256276033541878)
([(0.01158912, 'God'), (0.008576355, 'people'), (0.008176537, 'think'), (0.0075558797, 'know'), (0.0064504975, 'Jesus'), (0.006187106, 'believe'), (0.00576525, 'say')], -1.3865729483554852)
([(0.010859024, 'drive'), (0.010739261, 'card'), (0.009631535, 'use'), (0.009246996, 'problem'), (0.008278383, 'work'), (0.0075654243, 'system'), (0.0074819387, 'know')], -1.3908445482360863)
([(0.01128554, 'armenian'), (0.009634438, 'Ar

In [ ]:
train_log_perplexity = LDA.log_perplexity(bow_train_data)
test_log_perplexity = LDA.log_perplexity(bow_test_data)

In [243]:
print("Train log-perplexity score :" , round(train_log_perplexity,2))
print("Test log-perplexity score :" , round(test_log_perplexity,2))

Train log-perplexity score : -8.13
Test log-perplexity score : -8.49


In [240]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [241]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=LDA, texts= test_bigram, coherence='c_v' , topn=4)
coherence = coherence_model_lda.get_coherence()

In [242]:
print('Test Coherence Score: ', round(coherence , 2))

Test Coherence Score:  0.67


In [149]:
def get_LDA_vectors(bow_corpus):
  vectors = []
  for i in range(len(bow_corpus)):
      top_topics = LDA.get_document_topics(bow_corpus[i] , minimum_probability=0.0)
      topic_vec = [top_topics[i][1] for i in range(20)]
      vectors.append(topic_vec)
  return vectors

In [150]:
train_vectors = get_LDA_vectors(bow_train_data)
test_vectors = get_LDA_vectors(bow_test_data)

In [151]:
train_vectors[40]

[0.4337626,
 0.0005952382,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.0005952381,
 0.112444766,
 0.0005952381,
 0.15745255,
 0.2868163,
 0.0005952381,
 0.0005952382,
 0.0005952381,
 0.0005952381,
 0.0005952381]

In [153]:
train_labels = newsgroups_train.target
test_labels = newsgroups_test.target

In [152]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(train_vectors)
scaled_train = scalar.transform(train_vectors)
scaled_test = scalar.transform(test_vectors)

In [156]:
svc = SVC(verbose=5) 
params = {'C':[1,2,5,6], 'kernel' : ['rbf']}
model = GridSearchCV(svc, params , n_jobs= -1 ,verbose = 10,cv = 2)
model = model.fit(scaled_train,train_labels)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   17.9s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   23.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   23.8s finished


[LibSVM]

In [157]:
model.best_params_

{'C': 2, 'kernel': 'rbf'}

In [162]:
svc = SVC(C = 6 , kernel = 'rbf')
svc = svc.fit(scaled_train,train_labels)
pred_labels =svc.predict(scaled_test)

In [251]:
from sklearn.metrics import accuracy_score as accuracy
accuracy = accuracy(pred_labels,test_labels)
print('Accuracy :',accuracy)
print('f1_score :',metrics.f1_score(test_labels, pred_labels, average='weighted'))

Accuracy : 0.3835634625597451
f1_score : 0.3653025167225441


In [172]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

tfidf = TfidfVectorizer(analyzer= 'word', tokenizer=tokenizer)
tfidf_train = tfidf.fit_transform(newsgroups_train.data)
tfidf_test = tfidf.transform(newsgroups_test.data)

classifier_NB = MultinomialNB(alpha=0.01)
classifier_NB.fit(tfidf_train , train_labels)

pred = classifier_NB.predict(tfidf_test)

In [250]:
accuracy = accuracy(test_labels,pred)
print('Accuracy :',accuracy)
print('f1_score :',metrics.f1_score(test_labels, pred, average='weighted'))

Accuracy : 0.6872012745618693
f1_score : 0.6832558535521507
